### Import Necessary Module and Set Path 

In [1]:
import sqlite3 as lite
import os
import sys
import collections
sys.path.insert(0, '../../scripts/')
jupyter_directory = os.getcwd()

In [2]:
import analysis as an

## Run Cyclus Input file

### The output file should be named 
         
          eu_future.sqlite

In [3]:
# dictionary to hold data for plotting:
astrid_fuel_from_uox = {}
uox_reprocessed = {}
avg_reprocess_capacity = {}
max_reprocess_capacity = {}
uox_avg_reprocess_capacity = {}
uox_max_reprocess_capacity = {}

In [4]:
# get sqlite cursor and simulation time data
cur_108br = an.get_cursor('pref_astrid_mixer.sqlite')
cur_111br = an.get_cursor('111br.sqlite')
cur_115br = an.get_cursor('115br.sqlite')
cur_118br = an.get_cursor('118br.sqlite')
cur_5ext = an.get_cursor('5yr_108.sqlite')
cur_5ext_111br = an.get_cursor('5yr_111.sqlite')
cur_5ext_115br = an.get_cursor('5yr_115.sqlite')

init_year, init_month, duration, timestep = an.get_timesteps(cur_111br)

In [5]:
# Pu Composition in Spent Fuels:
pu_frac_spent_uox = cur_111br.execute('SELECT sum(massfrac) FROM recipes LEFT OUTER JOIN compositions ON '
                'compositions.qualid = recipes.qualid '
                'WHERE recipe = "uox_used_fuel_recipe" '
                'and (nucid > 940000000 and nucid < 950000000)').fetchone()
obr = cur_108br.execute('SELECT sum(massfrac) FROM recipes LEFT OUTER JOIN compositions ON '
                'compositions.qualid = recipes.qualid '
                'WHERE recipe = "sfr_spent_mox_recipe" '
                'and (nucid > 940000000 and nucid < 950000000)').fetchone()
fivebr = cur_115br.execute('SELECT sum(massfrac) FROM recipes LEFT OUTER JOIN compositions ON '
                'compositions.qualid = recipes.qualid '
                'WHERE recipe = "1.15" '
                'and (nucid > 940000000 and nucid < 950000000)').fetchone()
onebr = cur_111br.execute('SELECT sum(massfrac) FROM recipes LEFT OUTER JOIN compositions ON '
                'compositions.qualid = recipes.qualid '
                'WHERE recipe = "1.11" '
                'and (nucid > 940000000 and nucid < 950000000)').fetchone()
eightbr = cur_118br.execute('SELECT sum(massfrac) FROM recipes LEFT OUTER JOIN compositions ON '
                'compositions.qualid = recipes.qualid '
                'WHERE recipe = "1.18" '
                'and (nucid > 940000000 and nucid < 950000000)').fetchone()
print('Pu composition of spent fuel:')
print('BR 1.08:')
print(obr[0])
print('BR 1.11:')
print(onebr[0])
print('BR 1.15:')
print(fivebr[0])
print('BR 1.18:')
print(eightbr[0])

Pu composition of spent fuel:
BR 1.08:
0.27431405161777184
BR 1.11:
0.2927282490909536
BR 1.15:
0.3012337276327513
BR 1.18:
0.310254559737049


In [6]:
# 1.08 br case:
agent_ids = an.get_prototype_id(cur_108br, 'astrid_mixer_uox')
fuel_from_uox = an.facility_commodity_flux(cur_108br, agent_ids, ['astrid_fuel_legacy'], True, False)
agent_ids = an.get_prototype_id(cur_108br, 'astrid_mixer_astrid')
fuel_from_mox = an.facility_commodity_flux(cur_108br, agent_ids, ['astrid_fuel_new'], True, False)
dict = {}
dict['fuel from spent uox'] = fuel_from_uox['astrid_fuel_legacy']
dict['fuel from spent astrid_fuel'] = fuel_from_mox['astrid_fuel_new']
an.stacked_bar_chart(dict, timestep,
                  'Years', 'Mass[MTHM]',
                  'ASTRID Fuel Loading vs Time',
                  'where_fuel_sep',
                  init_year)
print('Total ASTRID fuel from spent UOX:')
print(sum(dict['fuel from spent uox']))
print('Total ASTRID fuel from spent ASTRID fuel:')
print(sum(dict['fuel from spent astrid_fuel']))

print('Total Legacy UNF Reprocessed:')
print(sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0])

astrid_fuel_from_uox['1.08'] = sum(dict['fuel from spent uox'])
uox_reprocessed['1.08'] = sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0]

# get pu flux and reprocessing capacity
agent_ids = an.get_prototype_id(cur_108br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_108br, agent_ids, ['uox_Pu'], True, False)
puflux = flux['uox_Pu']
total_pu = 0
months = 0

for i in range(0, len(puflux)):
    if puflux[i] != 0:
        total_pu += puflux[i]
        months += 1
print(total_pu)
print(months)

uox_avg_reprocess_capacity['1.08'] = total_pu / months / pu_frac_spent_uox[0]
uox_max_reprocess_capacity['1.08'] = max(puflux) / pu_frac_spent_uox[0]

print('Average Pu flux:')
print(total_pu / months)
print('Average reprocessing Capacity per month')
print(total_pu / months / pu_frac_spent_uox[0])
print('Maximum reprocessing Capcity reqd')
print(max(puflux) / pu_frac_spent_uox[0])



agent_ids = an.get_prototype_id(cur_108br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_108br, agent_ids, ['uox_Pu'], True, False)
agent_ids = an.get_prototype_id(cur_108br, 'astrid_reprocessing')
flux2 = an.facility_commodity_flux(cur_108br, agent_ids, ['astrid_Pu'], True, False)
puflux = flux['uox_Pu']
puflux2 = flux2['astrid_Pu']
rep_flux = [x / pu_frac_spent_uox[0] for x in puflux]
rep_flux2 = [x / obr[0] for x in puflux2]

tot_repflux = [x+y for x, y in zip(rep_flux, rep_flux2)]

total_rep = 0
months = 0 
for i in range(0, len(tot_repflux)):
    if tot_repflux[i] != 0:
        total_rep += tot_repflux[i]
        months += 1
print(total_rep)
print(months)

print('Maximum total reprocessing Capcity')
print(max(tot_repflux))
print('Average total reprocessing Capacity')
print(total_rep / months)


avg_reprocess_capacity['1.08'] = total_rep / months
max_reprocess_capacity['1.08'] = max(tot_repflux) 

/home/dkadkf/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1710: MatplotlibDeprecationWarning: The *left* kwarg to `bar` is deprecated use *x* instead. Support for *left* will be removed in Matplotlib 3.0
  return func(ax, *args, **kwargs)


Total ASTRID fuel from spent UOX:
2923.2
Total ASTRID fuel from spent ASTRID fuel:
60529.728
Total Legacy UNF Reprocessed:
54111.0155124
1269.22294725
885
Average Pu flux:
1.43415022288
Average reprocessing Capacity per month
120.669946008
Maximum reprocessing Capcity reqd
128.426674509
157617.278722
1669
Maximum total reprocessing Capcity
284.018866163
Average total reprocessing Capacity
94.4381538176


In [7]:
# 1.11 br case:
agent_ids = an.get_prototype_id(cur_111br, 'astrid_mixer_uox')
fuel_from_uox = an.facility_commodity_flux(cur_111br, agent_ids, ['astrid_fuel_legacy'], True, False)
agent_ids = an.get_prototype_id(cur_111br, 'astrid_mixer_astrid')
fuel_from_mox = an.facility_commodity_flux(cur_111br, agent_ids, ['astrid_fuel_new'], True, False)
dict = {}
dict['fuel from spent uox'] = fuel_from_uox['astrid_fuel_legacy']
dict['fuel from spent astrid_fuel'] = fuel_from_mox['astrid_fuel_new']
an.stacked_bar_chart(dict, timestep,
                  'Years', 'Mass[MTHM]',
                  'ASTRID Fuel Loading vs Time',
                  'where_fuel_sep',
                  init_year)
print('Total ASTRID fuel from spent UOX:')
print(sum(dict['fuel from spent uox']))
print('Total ASTRID fuel from spent ASTRID fuel:')
print(sum(dict['fuel from spent astrid_fuel']))

print('Total Legacy UNF Reprocessed:')
print(sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0])


astrid_fuel_from_uox['1.11'] = sum(dict['fuel from spent uox'])
uox_reprocessed['1.11'] = sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0]

# get pu flux and reprocessing capacity
agent_ids = an.get_prototype_id(cur_111br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_111br, agent_ids, ['uox_Pu'], True, False)
puflux = flux['uox_Pu']
total_pu = 0
months = 0 
for i in range(0, len(puflux)):
    if puflux[i] != 0:
        total_pu += puflux[i]
        months += 1
print(total_pu)
print(months)

print('Average Pu flux:')
print(total_pu / months)
print('Average reprocessing Capacity per month')
print(total_pu / months / pu_frac_spent_uox[0])
print('Maximum reprocessing Capcity reqd')
print(max(puflux) / pu_frac_spent_uox[0])


uox_avg_reprocess_capacity['1.11'] = total_pu / months / pu_frac_spent_uox[0]
uox_max_reprocess_capacity['1.11'] = max(puflux) / pu_frac_spent_uox[0]

agent_ids = an.get_prototype_id(cur_111br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_111br, agent_ids, ['uox_Pu'], True, False)
agent_ids = an.get_prototype_id(cur_111br, 'astrid_reprocessing')
flux2 = an.facility_commodity_flux(cur_111br, agent_ids, ['astrid_Pu'], True, False)
puflux = flux['uox_Pu']
puflux2 = flux2['astrid_Pu']
rep_flux = [x / pu_frac_spent_uox[0] for x in puflux]
rep_flux2 = [x / onebr[0] for x in puflux2]

tot_repflux = [x+y for x, y in zip(rep_flux, rep_flux2)]

total_rep = 0
months = 0 
for i in range(0, len(tot_repflux)):
    if tot_repflux[i] != 0:
        total_rep += tot_repflux[i]
        months += 1
print(total_rep)
print(months)

print('Maximum total reprocessing Capcity')
print(max(tot_repflux))
print('Average total reprocessing Capacity')
print(total_rep / months)


avg_reprocess_capacity['1.11'] = total_rep / months
max_reprocess_capacity['1.11'] = max(tot_repflux) 

/home/dkadkf/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1710: MatplotlibDeprecationWarning: The *left* kwarg to `bar` is deprecated use *x* instead. Support for *left* will be removed in Matplotlib 3.0
  return func(ax, *args, **kwargs)


Total ASTRID fuel from spent UOX:
2778.432
Total ASTRID fuel from spent ASTRID fuel:
60796.992
Total Legacy UNF Reprocessed:
51431.231887
1269.22294725
885
Average Pu flux:
1.43415022288
Average reprocessing Capacity per month
120.669946008
Maximum reprocessing Capcity reqd
128.426674509
154621.01161
1647
Maximum total reprocessing Capcity
232.101404031
Average total reprocessing Capacity
93.8803956345


In [8]:
# 1.15 br case
agent_ids = an.get_prototype_id(cur_115br, 'astrid_mixer_uox')
fuel_from_uox = an.facility_commodity_flux(cur_115br, agent_ids, ['astrid_fuel_legacy'], True, False)
agent_ids = an.get_prototype_id(cur_115br, 'astrid_mixer_astrid')
fuel_from_mox = an.facility_commodity_flux(cur_115br, agent_ids, ['astrid_fuel_new'], True, False)
dict = {}
dict['fuel from spent uox'] = fuel_from_uox['astrid_fuel_legacy']
dict['fuel from spent astrid_fuel'] = fuel_from_mox['astrid_fuel_new']
an.stacked_bar_chart(dict, timestep,
                  'Years', 'Mass[MTHM]',
                  'ASTRID Fuel Loading vs Time',
                  'where_fuel_sep',
                  init_year)
print('Total ASTRID fuel from spent UOX:')
print(sum(dict['fuel from spent uox']))
print('Total ASTRID fuel from spent ASTRID fuel:')
print(sum(dict['fuel from spent astrid_fuel']))

print('Total Legacy UNF Reprocessed:')
print(sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0])



astrid_fuel_from_uox['1.15'] = sum(dict['fuel from spent uox'])
uox_reprocessed['1.15'] = sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0]

# get pu flux and reprocessing capacity
agent_ids = an.get_prototype_id(cur_115br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_115br, agent_ids, ['uox_Pu'], True, False)
puflux = flux['uox_Pu']
total_pu = 0
months = 0 
for i in range(0, len(puflux)):
    if puflux[i] != 0:
        total_pu += puflux[i]
        months += 1
print(total_pu)
print(months)

print('Average Pu flux:')
print(total_pu / months)
print('Average reprocessing Capacity per month')
print(total_pu / months / pu_frac_spent_uox[0])
print('Maximum reprocessing Capcity reqd')
print(max(puflux) / pu_frac_spent_uox[0])


uox_avg_reprocess_capacity['1.15'] = total_pu / months / pu_frac_spent_uox[0]
uox_max_reprocess_capacity['1.15'] = max(puflux) / pu_frac_spent_uox[0]

agent_ids = an.get_prototype_id(cur_115br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_115br, agent_ids, ['uox_Pu'], True, False)
agent_ids = an.get_prototype_id(cur_115br, 'astrid_reprocessing')
flux2 = an.facility_commodity_flux(cur_115br, agent_ids, ['astrid_Pu'], True, False)
puflux = flux['uox_Pu']
puflux2 = flux2['astrid_Pu']
rep_flux = [x / pu_frac_spent_uox[0] for x in puflux]
rep_flux2 = [x / fivebr[0] for x in puflux2]

tot_repflux = [x+y for x, y in zip(rep_flux, rep_flux2)]

total_rep = 0
months = 0 
for i in range(0, len(tot_repflux)):
    if tot_repflux[i] != 0:
        total_rep += tot_repflux[i]
        months += 1
print(total_rep)
print(months)

print('Maximum total reprocessing Capcity')
print(max(tot_repflux))
print('Average total reprocessing Capacity')
print(total_rep / months)


avg_reprocess_capacity['1.15'] = total_rep / months
max_reprocess_capacity['1.15'] = max(tot_repflux) 

/home/dkadkf/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1710: MatplotlibDeprecationWarning: The *left* kwarg to `bar` is deprecated use *x* instead. Support for *left* will be removed in Matplotlib 3.0
  return func(ax, *args, **kwargs)


Total ASTRID fuel from spent UOX:
2694.912
Total ASTRID fuel from spent ASTRID fuel:
60964.032
Total Legacy UNF Reprocessed:
49885.2028723
1269.22294725
885
Average Pu flux:
1.43415022288
Average reprocessing Capacity per month
120.669946008
Maximum reprocessing Capcity reqd
128.426674509
153392.556347
1648
Maximum total reprocessing Capcity
237.658268031
Average total reprocessing Capacity
93.078007492


In [9]:
# 1.18 br case
agent_ids = an.get_prototype_id(cur_118br, 'astrid_mixer_uox')
fuel_from_uox = an.facility_commodity_flux(cur_118br, agent_ids, ['astrid_fuel_legacy'], True, False)
agent_ids = an.get_prototype_id(cur_118br, 'astrid_mixer_astrid')
fuel_from_mox = an.facility_commodity_flux(cur_118br, agent_ids, ['astrid_fuel_new'], True, False)
dict = {}
dict['fuel from spent uox'] = fuel_from_uox['astrid_fuel_legacy']
dict['fuel from spent astrid_fuel'] = fuel_from_mox['astrid_fuel_new']
an.stacked_bar_chart(dict, timestep,
                  'Years', 'Mass[MTHM]',
                  'ASTRID Fuel Loading vs Time',
                  'where_fuel_sep',
                  init_year)
print('Total ASTRID fuel from spent UOX:')
print(sum(dict['fuel from spent uox']))
print('Total ASTRID fuel from spent ASTRID fuel:')
print(sum(dict['fuel from spent astrid_fuel']))

print('Total Legacy UNF Reprocessed:')
print(sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0])



astrid_fuel_from_uox['1.18'] = sum(dict['fuel from spent uox'])
uox_reprocessed['1.18'] = sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0]

# get pu flux and reprocessing capacity
agent_ids = an.get_prototype_id(cur_118br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_118br, agent_ids, ['uox_Pu'], True, False)
puflux = flux['uox_Pu']
total_pu = 0
months = 0 
for i in range(0, len(puflux)):
    if puflux[i] != 0:
        total_pu += puflux[i]
        months += 1
print(total_pu)
print(months)

print('Average Pu flux:')
print(total_pu / months)
print('Average reprocessing Capacity per month')
print(total_pu / months / pu_frac_spent_uox[0])
print('Maximum reprocessing Capcity reqd')
print(max(puflux) / pu_frac_spent_uox[0])


uox_avg_reprocess_capacity['1.18'] = total_pu / months / pu_frac_spent_uox[0]
uox_max_reprocess_capacity['1.18'] = max(puflux) / pu_frac_spent_uox[0]

agent_ids = an.get_prototype_id(cur_118br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_118br, agent_ids, ['uox_Pu'], True, False)
agent_ids = an.get_prototype_id(cur_118br, 'astrid_reprocessing')
flux2 = an.facility_commodity_flux(cur_118br, agent_ids, ['astrid_Pu'], True, False)
puflux = flux['uox_Pu']
puflux2 = flux2['astrid_Pu']
rep_flux = [x / pu_frac_spent_uox[0] for x in puflux]
rep_flux2 = [x / eightbr[0] for x in puflux2]

tot_repflux = [x+y for x, y in zip(rep_flux, rep_flux2)]

total_rep = 0
months = 0 
for i in range(0, len(tot_repflux)):
    if tot_repflux[i] != 0:
        total_rep += tot_repflux[i]
        months += 1
print(total_rep)
print(months)

print('Maximum total reprocessing Capcity')
print(max(tot_repflux))
print('Average total reprocessing Capacity')
print(total_rep / months)


avg_reprocess_capacity['1.18'] = total_rep / months
max_reprocess_capacity['1.18'] = max(tot_repflux) 

/home/dkadkf/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1710: MatplotlibDeprecationWarning: The *left* kwarg to `bar` is deprecated use *x* instead. Support for *left* will be removed in Matplotlib 3.0
  return func(ax, *args, **kwargs)


Total ASTRID fuel from spent UOX:
2639.232
Total ASTRID fuel from spent ASTRID fuel:
61092.096
Total Legacy UNF Reprocessed:
48854.5168626
1269.22294725
885
Average Pu flux:
1.43415022288
Average reprocessing Capacity per month
120.669946008
Maximum reprocessing Capcity reqd
128.426674509
152128.45381
1643
Maximum total reprocessing Capcity
234.00707368
Average total reprocessing Capacity
92.5918769385


In [10]:
# 5 yr ext case
agent_ids = an.get_prototype_id(cur_5ext, 'astrid_mixer_uox')
fuel_from_uox = an.facility_commodity_flux(cur_5ext, agent_ids, ['astrid_fuel_legacy'], True, False)
agent_ids = an.get_prototype_id(cur_5ext, 'astrid_mixer_astrid')
fuel_from_mox = an.facility_commodity_flux(cur_5ext, agent_ids, ['astrid_fuel_new'], True, False)
dict = {}
dict['fuel from spent uox'] = fuel_from_uox['astrid_fuel_legacy']
dict['fuel from spent astrid_fuel'] = fuel_from_mox['astrid_fuel_new']
an.stacked_bar_chart(dict, timestep,
                  'Years', 'Mass[MTHM]',
                  'ASTRID Fuel Loading vs Time',
                  'where_fuel_sep',
                  init_year)
print('Total ASTRID fuel from spent UOX:')
print(sum(dict['fuel from spent uox']))
print('Total ASTRID fuel from spent ASTRID fuel:')
print(sum(dict['fuel from spent astrid_fuel']))

print('Total Legacy UNF Reprocessed:')
print(sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0])


astrid_fuel_from_uox['5ext'] = sum(dict['fuel from spent uox'])
uox_reprocessed['5ext'] = sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0]


# get pu flux and reprocessing capacity
agent_ids = an.get_prototype_id(cur_5ext, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_5ext, agent_ids, ['uox_Pu'], True, False)
puflux = flux['uox_Pu']
total_pu = 0
months = 0 
for i in range(0, len(puflux)):
    if puflux[i] != 0:
        total_pu += puflux[i]
        months += 1
print(total_pu)
print(months)

print('Average Pu flux:')
print(total_pu / months)
print('Average reprocessing Capacity per month')
print(total_pu / months / pu_frac_spent_uox[0])
print('Maximum reprocessing Capcity reqd')
print(max(puflux) / pu_frac_spent_uox[0])


uox_avg_reprocess_capacity['5ext'] = total_pu / months / pu_frac_spent_uox[0]
uox_max_reprocess_capacity['5ext'] = max(puflux) / pu_frac_spent_uox[0]

agent_ids = an.get_prototype_id(cur_5ext, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_5ext, agent_ids, ['uox_Pu'], True, False)
agent_ids = an.get_prototype_id(cur_5ext, 'astrid_reprocessing')
flux2 = an.facility_commodity_flux(cur_5ext, agent_ids, ['astrid_Pu'], True, False)
puflux = flux['uox_Pu']
puflux2 = flux2['astrid_Pu']
rep_flux = [x / pu_frac_spent_uox[0] for x in puflux]
rep_flux2 = [x / obr[0] for x in puflux2]

tot_repflux = [x+y for x, y in zip(rep_flux, rep_flux2)]

total_rep = 0
months = 0 
for i in range(0, len(tot_repflux)):
    if tot_repflux[i] != 0:
        total_rep += tot_repflux[i]
        months += 1
print(total_rep)
print(months)

print('Maximum total reprocessing Capcity')
print(max(tot_repflux))
print('Average total reprocessing Capacity')
print(total_rep / months)


avg_reprocess_capacity['5ext'] = total_rep / months
max_reprocess_capacity['5ext'] = max(tot_repflux) 

/home/dkadkf/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1710: MatplotlibDeprecationWarning: The *left* kwarg to `bar` is deprecated use *x* instead. Support for *left* will be removed in Matplotlib 3.0
  return func(ax, *args, **kwargs)


Total ASTRID fuel from spent UOX:
3012.288
Total ASTRID fuel from spent ASTRID fuel:
41047.296
Total Legacy UNF Reprocessed:
55760.113128
1321.09916272
923
Average Pu flux:
1.43131003544
Average reprocessing Capacity per month
120.430971555
Maximum reprocessing Capcity reqd
128.426674509
146410.829974
1379
Maximum total reprocessing Capcity
217.336459653
Average total reprocessing Capacity
106.171740373


In [11]:
# 5ext + 1.11 br case
agent_ids = an.get_prototype_id(cur_5ext_111br, 'astrid_mixer_uox')
fuel_from_uox = an.facility_commodity_flux(cur_5ext_111br, agent_ids, ['astrid_fuel_legacy'], True, False)
agent_ids = an.get_prototype_id(cur_5ext_111br, 'astrid_mixer_astrid')
fuel_from_mox = an.facility_commodity_flux(cur_5ext_111br, agent_ids, ['astrid_fuel_new'], True, False)
dict = {}
dict['fuel from spent uox'] = fuel_from_uox['astrid_fuel_legacy']
dict['fuel from spent astrid_fuel'] = fuel_from_mox['astrid_fuel_new']
an.stacked_bar_chart(dict, timestep,
                  'Years', 'Mass[MTHM]',
                  'ASTRID Fuel Loading vs Time',
                  'where_fuel_sep',
                  init_year)
print('Total ASTRID fuel from spent UOX:')
print(sum(dict['fuel from spent uox']))
print('Total ASTRID fuel from spent ASTRID fuel:')
print(sum(dict['fuel from spent astrid_fuel']))

print('Total Legacy UNF Reprocessed:')
print(sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0])



astrid_fuel_from_uox['5ext_1.11'] = sum(dict['fuel from spent uox'])
uox_reprocessed['5ext_1.11'] = sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0]


# get pu flux and reprocessing capacity
agent_ids = an.get_prototype_id(cur_5ext_111br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_5ext_111br, agent_ids, ['uox_Pu'], True, False)
puflux = flux['uox_Pu']
total_pu = 0
months = 0 
for i in range(0, len(puflux)):
    if puflux[i] != 0:
        total_pu += puflux[i]
        months += 1
print(total_pu)
print(months)

print('Average Pu flux:')
print(total_pu / months)
print('Average reprocessing Capacity per month')
print(total_pu / months / pu_frac_spent_uox[0])
print('Maximum reprocessing Capcity reqd')
print(max(puflux) / pu_frac_spent_uox[0])


uox_avg_reprocess_capacity['5ext_1.11'] = total_pu / months / pu_frac_spent_uox[0]
uox_max_reprocess_capacity['5ext_1.11'] = max(puflux) / pu_frac_spent_uox[0]

agent_ids = an.get_prototype_id(cur_5ext_111br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_5ext_111br, agent_ids, ['uox_Pu'], True, False)
agent_ids = an.get_prototype_id(cur_5ext_111br, 'astrid_reprocessing')
flux2 = an.facility_commodity_flux(cur_5ext_111br, agent_ids, ['astrid_Pu'], True, False)
puflux = flux['uox_Pu']
puflux2 = flux2['astrid_Pu']
rep_flux = [x / pu_frac_spent_uox[0] for x in puflux]
rep_flux2 = [x / onebr[0] for x in puflux2]

tot_repflux = [x+y for x, y in zip(rep_flux, rep_flux2)]

total_rep = 0
months = 0 
for i in range(0, len(tot_repflux)):
    if tot_repflux[i] != 0:
        total_rep += tot_repflux[i]
        months += 1
print(total_rep)
print(months)

print('Maximum total reprocessing Capcity')
print(max(tot_repflux))
print('Average total reprocessing Capacity')
print(total_rep / months)


avg_reprocess_capacity['5ext_1.11'] = total_rep / months
max_reprocess_capacity['5ext_1.11'] = max(tot_repflux) 

/home/dkadkf/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1710: MatplotlibDeprecationWarning: The *left* kwarg to `bar` is deprecated use *x* instead. Support for *left* will be removed in Matplotlib 3.0
  return func(ax, *args, **kwargs)


Total ASTRID fuel from spent UOX:
2873.088
Total ASTRID fuel from spent ASTRID fuel:
41192.064
Total Legacy UNF Reprocessed:
53183.3981036
1321.09916272
923
Average Pu flux:
1.43131003544
Average reprocessing Capacity per month
120.430971555
Maximum reprocessing Capcity reqd
128.426674509
144302.022432
1373
Maximum total reprocessing Capcity
224.527466628
Average total reprocessing Capacity
105.099797838


In [12]:
# 5 yr ext + 1.15 br case
agent_ids = an.get_prototype_id(cur_5ext_115br, 'astrid_mixer_uox')
fuel_from_uox = an.facility_commodity_flux(cur_5ext_115br, agent_ids, ['astrid_fuel_legacy'], True, False)
agent_ids = an.get_prototype_id(cur_5ext_115br, 'astrid_mixer_astrid')
fuel_from_mox = an.facility_commodity_flux(cur_5ext_115br, agent_ids, ['astrid_fuel_new'], True, False)
dict = {}
dict['fuel from spent uox'] = fuel_from_uox['astrid_fuel_legacy']
dict['fuel from spent astrid_fuel'] = fuel_from_mox['astrid_fuel_new']
an.stacked_bar_chart(dict, timestep,
                  'Years', 'Mass[MTHM]',
                  'ASTRID Fuel Loading vs Time',
                  'where_fuel_sep',
                  init_year)
print('Total ASTRID fuel from spent UOX:')
print(sum(dict['fuel from spent uox']))
print('Total ASTRID fuel from spent ASTRID fuel:')
print(sum(dict['fuel from spent astrid_fuel']))

print('Total Legacy UNF Reprocessed:')
print(sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0])

astrid_fuel_from_uox['5ext_1.15'] = sum(dict['fuel from spent uox'])
uox_reprocessed['5ext_1.15'] = sum(dict['fuel from spent uox'])*.22/pu_frac_spent_uox[0]

# get pu flux and reprocessing capacity
agent_ids = an.get_prototype_id(cur_5ext_115br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_5ext_115br, agent_ids, ['uox_Pu'], True, False)
puflux = flux['uox_Pu']
total_pu = 0
months = 0 
for i in range(0, len(puflux)):
    if puflux[i] != 0:
        total_pu += puflux[i]
        months += 1
print(total_pu)
print(months)

print('Average Pu flux:')
print(total_pu / months)
print('Average UOX reprocessing Capacity per month')
print(total_pu / months / pu_frac_spent_uox[0])
print('Maximum UOX reprocessing Capcity reqd')
print(max(puflux) / pu_frac_spent_uox[0])

uox_avg_reprocess_capacity['5ext_1.15'] = total_pu / months / pu_frac_spent_uox[0]
uox_max_reprocess_capacity['5ext_1.15'] = max(puflux) / pu_frac_spent_uox[0]

agent_ids = an.get_prototype_id(cur_5ext_115br, 'uox_reprocessing')
flux = an.facility_commodity_flux(cur_5ext_115br, agent_ids, ['uox_Pu'], True, False)
agent_ids = an.get_prototype_id(cur_5ext_115br, 'astrid_reprocessing')
flux2 = an.facility_commodity_flux(cur_5ext_115br, agent_ids, ['astrid_Pu'], True, False)
puflux = flux['uox_Pu']
puflux2 = flux2['astrid_Pu']
rep_flux = [x / pu_frac_spent_uox[0] for x in puflux]
rep_flux2 = [x / fivebr[0] for x in puflux2]

tot_repflux = [x+y for x, y in zip(rep_flux, rep_flux2)]

total_rep = 0
months = 0 
for i in range(0, len(tot_repflux)):
    if tot_repflux[i] != 0:
        total_rep += tot_repflux[i]
        months += 1
print(total_rep)
print(months)

print('Maximum total reprocessing Capcity')
print(max(tot_repflux))
print('Average total reprocessing Capacity')
print(total_rep / months)


avg_reprocess_capacity['5ext_1.15'] = total_rep / months
max_reprocess_capacity['5ext_1.15'] = max(tot_repflux) 

/home/dkadkf/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:1710: MatplotlibDeprecationWarning: The *left* kwarg to `bar` is deprecated use *x* instead. Support for *left* will be removed in Matplotlib 3.0
  return func(ax, *args, **kwargs)


Total ASTRID fuel from spent UOX:
2800.704
Total ASTRID fuel from spent ASTRID fuel:
41264.448
Total Legacy UNF Reprocessed:
51843.5062909
1321.09916272
923
Average Pu flux:
1.43131003544
Average UOX reprocessing Capacity per month
120.430971555
Maximum UOX reprocessing Capcity reqd
128.426674509
143419.043266
1373
Maximum total reprocessing Capcity
221.809899594
Average total reprocessing Capacity
104.456695751


In [13]:
#display them:
for key in astrid_fuel_from_uox:
    print(key)
    print('ASTIRD Fuel From UOX:')
    print(astrid_fuel_from_uox[key])
    print('UOX Reprocessed:')
    print(uox_reprocessed[key])
    print('Average UOX UNF Reprocessing Capacity')
    print(uox_avg_reprocess_capacity[key])
    print('Maximum UOX UNF Reprocessing Capacity')
    print(uox_max_reprocess_capacity[key])
    print('Average Total Reprocessing Capacity')
    print(avg_reprocess_capacity[key])
    print('Maximum Total Reprocessing Capacity')
    print(max_reprocess_capacity[key])
    print('\n')

1.11
ASTIRD Fuel From UOX:
2778.432
UOX Reprocessed:
51431.231887
Average UOX UNF Reprocessing Capacity
120.669946008
Maximum UOX UNF Reprocessing Capacity
128.426674509
Average Total Reprocessing Capacity
93.8803956345
Maximum Total Reprocessing Capacity
232.101404031


5ext_1.11
ASTIRD Fuel From UOX:
2873.088
UOX Reprocessed:
53183.3981036
Average UOX UNF Reprocessing Capacity
120.430971555
Maximum UOX UNF Reprocessing Capacity
128.426674509
Average Total Reprocessing Capacity
105.099797838
Maximum Total Reprocessing Capacity
224.527466628


5ext
ASTIRD Fuel From UOX:
3012.288
UOX Reprocessed:
55760.113128
Average UOX UNF Reprocessing Capacity
120.430971555
Maximum UOX UNF Reprocessing Capacity
128.426674509
Average Total Reprocessing Capacity
106.171740373
Maximum Total Reprocessing Capacity
217.336459653


5ext_1.15
ASTIRD Fuel From UOX:
2800.704
UOX Reprocessed:
51843.5062909
Average UOX UNF Reprocessing Capacity
120.430971555
Maximum UOX UNF Reprocessing Capacity
128.426674509
Av